In [20]:
%%writefile testp.cpp
#include <bits/stdc++.h>
#include <omp.h>

using namespace std;

long long int nrOfSolutions=0;
int size=0;
int nrOdd = 0;
long long int arr[500];
int LEVEL = 3;
int cnt[20] ={0};

void setQueen(int queens[], int row, int col, int id, int col1) 
{
    for(int i=0; i<row; i++) 
    {
        // vertical attacks
        if (queens[i]==col) 
        {
            return;
        }
        // diagonal attacks
        if (abs(queens[i]-col) == (row-i) ) 
        {
            return;
        }
    }

queens[row]=col;
if(row==size-1 && col1!=(size/2)) {
    arr[omp_get_thread_num()]++;
}
else if(row==size-1 && col1==(size/2)){
    nrOdd++;
}
else{
    for(int i=0; i<size; i++) {
        setQueen(queens, row+1, i, id,col1);
        }
    }
}

void setQueen1(int queens[], int row, int col, int id, int col1) {
for(int i=0; i<row; i++) {
// vertical attacks
if (queens[i]==col) {
return;
}
// diagonal attacks
if (abs(queens[i]-col) == (row-i) ) {
return;
}
}

queens[row]=col;
if(row==size-1 && col1!=(size/2)) {
arr[omp_get_thread_num()]++;
}
else if(row==size-1 && col1==(size/2)){
nrOdd++;
}
else {
if(row<=LEVEL){
for(int i=0; i<size; i++) {
#pragma omp parallel
#pragma omp single
{
#pragma omp task
setQueen1(queens, row+1, i, id,col1);
}
}
}
else{
for(int i=0; i<size; i++) {
setQueen1(queens, row+1, i, id,col1);
}
}
}
}

void solve() {
int myid=0 ;

if(size%2==0){
#pragma omp parallel
#pragma omp single
{
for(int i=0; i<(size/2)-3; i++) {
#pragma omp task
setQueen(new int[size], 0, i, myid,i);
}
for(int i=(size/2)-3;i<(size/2);i++){
setQueen1(new int[size], 0, i, myid,i);
}
}

}
else{
#pragma omp parallel
#pragma omp single
{
for(int i=0; i<=(size/2); i++) {
#pragma omp task
setQueen(new int[size], 0, i, myid,i);
}
}
}

}

int main(int argc, char*argv[]) {

int queens = 8;
size = queens;
for(int i=0;i<500;i++){
arr[i] = 0;
}
double sTime = omp_get_wtime();
solve();
double eTime = omp_get_wtime();
long long int ans = 0;
int mn = 1000;
for(int i=0;i<500;i++){
if(arr[i]==0){
mn = min(mn,i);
}
nrOfSolutions += arr[i];
}
if(size%2==0){
ans = nrOfSolutions*2;
}
else{
ans = (nrOfSolutions*2)+nrOdd;
}
cout<<ans<<endl;
cout<<"Time taken for execusion is :"<<(eTime-sTime)<<endl;
return 0;
}


Overwriting testp.cpp


In [21]:
%%script bash
g++ -fopenmp -o nqu testp.cpp
./nqu

92
Time taken for execusion is :0.0026811


In [22]:
%%writefile seq.cpp
#include<chrono>
#include <omp.h>
#include<bits/stdc++.h>
#define MAX_N 8
using namespace std;
using namespace std::chrono;
bool isSafe(vector<vector<string> >& board, int row, int col, int N)
{
//check left side rows
for (auto j = 0; j<col; ++j)
if (board[row][j] == "Q")
return false;
//check top left side diagonal
for (auto i = row, j = col; i>-1 && j>-1; --i, --j)
if (board[i][j] == "Q")
return false;
//check bottom left side diagonal
for (auto i = row, j = col; i<N && j>-1; ++i, --j)
if (board[i][j] == "Q")
return false;
return true;
}
bool backtracking(vector<vector<string> >& board, int col, vector<vector<string> >& testboard, int N)
{
if (col == N)
{
vector<string> emptyRow;
board.emplace_back(emptyRow);
int size = board.size();
for (auto i = 0; i<N; ++i)
{
string row = "";
for (auto j = 0; j<N; ++j)
row += testboard[i][j];
board[size-1].emplace_back(row);
}
return false;
}
for (auto i = 0; i<N; ++i)
{
if (isSafe(testboard, i, col, N))
{
testboard[i][col] = "Q";
if (!backtracking(board, col+1, testboard, N))
testboard[i][col] = ".";
else
return true;
}
}
return false;
}
vector<vector<string> > solveNQueens(int A) {
vector<vector<string> > testboard;
if (A==2 || A==3)
return testboard;
vector<string> row(A, ".");
for (auto i = 0; i<A; ++i)
testboard.emplace_back(row);
vector<vector<string> > board;
backtracking(board, 0, testboard, A);
return board;
}

int main(){
int cnt=0;
//auto start = high_resolution_clock::now();
double sTime = omp_get_wtime();
vector<vector<string>> ans = solveNQueens(8);
double eTime = omp_get_wtime();
for( auto const& string_vec : ans )
{
for( auto const& s : string_vec )
{
cout << s << endl;
}
cnt++;
cout<<"==============\n";
}
// auto stop = high_resolution_clock::now();
cout<<"Time taken for execusion is :"<<(eTime-sTime)<<endl;
// Get duration. Substart timepoints to
// get durarion. To cast it to proper unit
// use duration cast method
// auto duration = duration_cast<milliseconds>(stop - start);
//cout << "Time taken by function: "
//<< duration.count() << " milliseconds" << endl;
cout<<cnt;
}


Overwriting seq.cpp


In [23]:
%%script bash
g++ -fopenmp -o myseq seq.cpp
./myseq

Q.......
......Q.
....Q...
.......Q
.Q......
...Q....
.....Q..
..Q.....
Q.......
......Q.
...Q....
.....Q..
.......Q
.Q......
....Q...
..Q.....
Q.......
.....Q..
.......Q
..Q.....
......Q.
...Q....
.Q......
....Q...
Q.......
....Q...
.......Q
.....Q..
..Q.....
......Q.
.Q......
...Q....
.....Q..
Q.......
....Q...
.Q......
.......Q
..Q.....
......Q.
...Q....
...Q....
Q.......
....Q...
.......Q
.Q......
......Q.
..Q.....
.....Q..
....Q...
Q.......
.......Q
...Q....
.Q......
......Q.
..Q.....
.....Q..
..Q.....
Q.......
......Q.
....Q...
.......Q
.Q......
...Q....
.....Q..
....Q...
Q.......
...Q....
.....Q..
.......Q
.Q......
......Q.
..Q.....
......Q.
Q.......
..Q.....
.......Q
.....Q..
...Q....
.Q......
....Q...
....Q...
Q.......
.......Q
.....Q..
..Q.....
......Q.
.Q......
...Q....
...Q....
Q.......
....Q...
.......Q
.....Q..
..Q.....
......Q.
.Q......
.Q......
.....Q..
Q.......
......Q.
...Q....
.......Q
..Q.....
....Q...
....Q...
..Q.....
Q.......
......Q.
.Q......
.......Q
.....Q..
.